## Data Pre-processing

In [49]:
# import packages
import numpy as np
np.random.seed(3939)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2 # Used for image augmentation

# from fancyimpute import KNN # Used for imputing missing data
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import compute_class_weight
from sklearn.metrics import log_loss
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model


In [50]:
# Reproducible random seed
seed = 6329

In [51]:
# load data
train = pd.read_json('train_imputed.json')
test = pd.read_json('test.json')

# convert inc_angle to numeric
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')

In [52]:
# Number of training observations
len(train)

1604

In [53]:
# Observe data
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
10,"[-21.397552, -19.753859, -23.426783, -24.65221...","[-26.72291, -27.418192, -27.787899, -25.774536...",3aac67cd,44.6240,1
100,"[-20.04884, -19.469616, -20.510244, -19.61095,...","[-29.742329, -26.374287, -25.490265, -25.49031...",66348d03,41.1342,0
1000,"[-23.199345, -23.603487, -25.965549, -27.12546...","[-23.004148, -24.942425, -24.472878, -23.00437...",7052a617,33.8975,0


In [54]:
test.head()

,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.966400
1,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",4023181e,32.615072
2,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",b20200e4,37.505433
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.473900
4,"[-23.37459373474121, -26.02718162536621, -28.1...","[-25.72234344482422, -27.011577606201172, -23....",4371c8c3,43.918874


In [9]:
# Inspect rows for missing inc_angle values
trn_nangle_locs = train.loc[train['inc_angle'].isnull()].index
print("Found", len(trn_nangle_locs), "training observations with missing inc_angle data.")

Found 0 training observations with missing inc_angle data.


In [10]:
tst_nangle_locs = test.loc[test['inc_angle'].isnull()].index
print("Found", len(tst_nangle_locs), "testing observations with missing inc_angle data.")

Found 0 testing observations with missing inc_angle data.


In [9]:
# Fill in missing values using the fancyimpute package

# Make a copy of the training set
train_copy = train.copy()

# Make dataframes of pixel values for each band
band_1_df = pd.DataFrame(i for i in train_copy['band_1'])
band_2_df = pd.DataFrame(i for i in train_copy['band_2'])

# Combine to make dataframe for autocompletion
train_copy_df = pd.concat([band_1_df, band_2_df, train_copy['inc_angle']], axis=1)

# Retain column names before procesing
train_copy_df_cols = list(train_copy_df)

# Perform the K nearest neighbor imputation for the missing values
train_copy_df = pd.DataFrame(KNN(k=3).fit_transform(train_copy_df))

# Return column names to the dataframe
train_copy_df.columns = train_copy_df_cols


Imputing row 1/1604 with 0 missing, elapsed time: 154.505
Imputing row 101/1604 with 0 missing, elapsed time: 154.514
Imputing row 201/1604 with 0 missing, elapsed time: 154.520
Imputing row 301/1604 with 0 missing, elapsed time: 154.524
Imputing row 401/1604 with 0 missing, elapsed time: 154.528
Imputing row 501/1604 with 0 missing, elapsed time: 154.533
Imputing row 601/1604 with 0 missing, elapsed time: 154.536
Imputing row 701/1604 with 0 missing, elapsed time: 154.541
Imputing row 801/1604 with 0 missing, elapsed time: 154.546
Imputing row 901/1604 with 0 missing, elapsed time: 154.550
Imputing row 1001/1604 with 0 missing, elapsed time: 154.554
Imputing row 1101/1604 with 0 missing, elapsed time: 154.558
Imputing row 1201/1604 with 0 missing, elapsed time: 154.563
Imputing row 1301/1604 with 0 missing, elapsed time: 154.567
Imputing row 1401/1604 with 0 missing, elapsed time: 154.571
Imputing row 1501/1604 with 0 missing, elapsed time: 154.574
Imputing row 1601/1604 with 1 missin

In [10]:
# Put completed inc_angle values into original dataframe
train['inc_angle'] = train_copy_df['inc_angle']

# Confirm no values are missing
trn_nangle_locs = train.loc[train['inc_angle'].isnull()].index
print("Found", len(trn_nangle_locs), "training observations with missing inc_angle data after imputing.")


Found 0 training observations with missing inc_angle data after imputing.


In [11]:
# Rescale data for image classifier
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [12]:
# Generate training images and labels
Xtrain = get_scaled_imgs(train)
Ytrain = np.array(train['is_iceberg'])

Xtest = get_scaled_imgs(test)


In [13]:
# Extract angle array for train, test
Xtrain_angle = train['inc_angle']
Xtest_angle = test['inc_angle']

# extract training targets
Ytrain = train['is_iceberg']

In [14]:
# Inspect image dimensions
Xtrain.shape[1:]

(75, 75, 3)

In [15]:
# Reshape band_1 data for visualisation
Xtrain_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])

## Data Visualisation

In [15]:
# prepare for visualizations
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

def plotmy3d(c, name):

    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=400,
        height=400,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [25]:
# look at an iceberg
plotmy3d(Xtrain_band_1[5,:,:], 'Iceberg')

In [26]:
# look at a ship
plotmy3d(Xtrain_band_1[17,:,:], 'Ship')

## Benchmark Model

In [25]:
def bmarkModel():
    
    # build keras model
    model = Sequential()
    
    # 1st conv. layer
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=Xtrain.shape[1:]))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))
    
    # 2nd conv. layer
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))
    
    # 3rd conv. layer
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))
    
    # 4th conv. layer
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))
    
    # flatten data for dense layers
    model.add(Flatten())

    # 1st dense layer
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    # 2nd dense layer
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # output 
    model.add(Dense(1, activation="sigmoid"))

    optimizer = "adam"
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [27]:
# prepare the model
bmark_model = bmarkModel()
bmark_model.summary()

batch_size = 32
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
bmark_mcp_save = ModelCheckpoint('.bmark_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [28]:
# train the model
bmark_model.fit(Xtrain, Ytrain, batch_size=batch_size, epochs=50, verbose=1, \
          callbacks=[earlyStopping, bmark_mcp_save, reduce_lr_loss], validation_split=0.25)

Train on 1203 samples, validate on 401 samples
Epoch 1/50
1203/1203 [==============================] - 5s - loss: 0.6855 - acc: 0.5295 - val_loss: 0.6427 - val_acc: 0.4913
Epoch 2/50
1203/1203 [==============================] - 1s - loss: 0.6690 - acc: 0.5495 - val_loss: 0.6965 - val_acc: 0.4913
Epoch 3/50
1203/1203 [==============================] - 1s - loss: 0.6537 - acc: 0.6185 - val_loss: 0.5316 - val_acc: 0.7531
Epoch 4/50
1203/1203 [==============================] - 1s - loss: 0.5196 - acc: 0.7415 - val_loss: 0.4311 - val_acc: 0.8304
Epoch 5/50
1203/1203 [==============================] - 1s - loss: 0.4345 - acc: 0.7930 - val_loss: 0.3778 - val_acc: 0.8379
Epoch 6/50
1203/1203 [==============================] - 1s - loss: 0.4064 - acc: 0.8047 - val_loss: 0.5987 - val_acc: 0.7007
Epoch 7/50
1203/1203 [==============================] - 1s - loss: 0.3672 - acc: 0.8196 - val_loss: 0.3250 - val_acc: 0.8554
Epoch 8/50
1203/1203 [==============================] - 1s - loss: 0.3144 - ac

In [29]:
# load best weights and score training data
bmark_model.load_weights(filepath = '.bmark_mdl_wts.hdf5')

score = bmark_model.evaluate(Xtrain, Ytrain, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

1568/1604 [============================>.] - ETA: 0sTrain score: 0.145048794216
Train accuracy: 0.94825436409


In [30]:
# Make predictions for test set
bmark_pred_test = bmark_model.predict(Xtest)

# Prepare submission
bmark_submission = pd.DataFrame({'id': test["id"], 'is_iceberg': bmark_pred_test.reshape((bmark_pred_test.shape[0]))})
print(bmark_submission.head(10))

# Export as csv
bmark_submission.to_csv('submissions/bmark_submission.csv', index=False)

         id  is_iceberg
0  5941774d    0.048651
1  4023181e    0.725734
2  b20200e4    0.007621
3  e7f018bb    0.996181
4  4371c8c3    0.821735
5  a8d9b1fd    0.445165
6  29e7727e    0.005876
7  92a51ffb    0.998043
8  c769ac97    0.000043
9  aee0547d    0.000754


In [38]:
# Capture benchmark model architecture
plot_model(bmark_model, to_file='model_arch/bmark_model.jpg', show_shapes=True, show_layer_names=True)

## Solution Model

In [39]:
# Generate additional augmented images from existing data
def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [40]:
# Generate extra training data
Xtr_more = get_more_images(Xtrain) 
Ytr_more = np.concatenate((Ytrain,Ytrain,Ytrain))

# Estimate class weights since the dataset is unbalanced
class_weights = dict(zip([0, 1], compute_class_weight('balanced', [0, 1], Ytr_more)))

In [41]:
# Create train/test indices to split data in train/test sets
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

In [62]:
# Connect transfer learning network
def tlearnModel():
    
    base_model = VGG16(weights='imagenet', include_top=False,
                       input_shape=Xtrain.shape[1:], classes=1)
    
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D()(x)
    #x = GlobalAveragePooling2D()(x)
    #x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    adam = Adam(lr=1e-3, decay=0.0)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    return model

In [43]:
model = tlearnModel()
model.summary()

batch_size = 8
earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.tlearn_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')


k = 0

y_train_pred_log = 0
y_valid_pred_log = 0.0 * Ytr_more
true_test_predictions = 0

for train, test in kfold.split(Xtr_more, Ytr_more):
    print('\n\t\tFOLD', k+1, 'of', n_splits, '\n')
    
    # Define model
    model = tlearnModel()
    
    # Fit the model
    history = model.fit(Xtr_more[train], Ytr_more[train],
                        batch_size=batch_size,
                        epochs=50,
                        verbose=1,
                        shuffle=True,
                        validation_data=(Xtr_more[test], Ytr_more[test]),
                        class_weight=class_weights,
                        callbacks=[earlyStopping, mcp_save, reduce_lr_loss])
    
    # Getting the Best Model
    model.load_weights(filepath='.tlearn_mdl_wts.hdf5')
    
    #Getting Training Score
    score = model.evaluate(Xtr_more[train], Ytr_more[train], verbose=0)
    print('Train loss:', score[0])
    print('Train accuracy:', score[1])
    #Getting Test Score
    score = model.evaluate(Xtr_more[test], Ytr_more[test], verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    #Getting Train Scores
    pred_train = model.predict(Xtr_more)
    y_train_pred_log += pred_train.reshape(pred_train.shape[0])
    
    #Getting validation Score
    pred_valid = model.predict(Xtr_more[test])
    y_valid_pred_log[test] = pred_valid.reshape(pred_valid.shape[0])
    
    # Predict True Test Score
    pred_test = model.predict(Xtest)
    true_test_predictions += pred_test.reshape(pred_test.shape[0])
    
    
    k += 1

true_test_predictions = true_test_predictions / n_splits
y_train_pred_log = y_train_pred_log / n_splits

print('\n Train Log Loss Validation= ', log_loss(Ytr_more, y_train_pred_log))
print(' Test Log Loss Validation= ', log_loss(Ytr_more, y_valid_pred_log))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 18, 128)       0         
__________

3849/3849 [==============================] - 46s - loss: 0.5214 - acc: 0.7191 - val_loss: 0.3336 - val_acc: 0.8359
Epoch 2/50
3849/3849 [==============================] - 46s - loss: 0.3114 - acc: 0.8488 - val_loss: 0.2595 - val_acc: 0.8795
Epoch 3/50
3849/3849 [==============================] - 46s - loss: 0.2532 - acc: 0.8953 - val_loss: 0.2243 - val_acc: 0.8982
Epoch 4/50
3849/3849 [==============================] - 46s - loss: 0.2218 - acc: 0.9023 - val_loss: 0.2394 - val_acc: 0.8910
Epoch 5/50
3849/3849 [==============================] - 46s - loss: 0.1958 - acc: 0.9226 - val_loss: 0.3438 - val_acc: 0.8567
Epoch 6/50
3849/3849 [==============================] - 46s - loss: 0.1837 - acc: 0.9223 - val_loss: 0.2184 - val_acc: 0.9065
Epoch 7/50
3849/3849 [==============================] - 46s - loss: 0.1540 - acc: 0.9402 - val_loss: 0.3534 - val_acc: 0.8775
Epoch 8/50
3849/3849 [==============================] - 46s - loss: 0.1425 - acc: 0.9413 - val_loss: 0.2754 - val_acc: 0.8951
Epo

In [55]:
# Generate dataframe of predictions for submission

tlearn_submission = pd.DataFrame({'id': test["id"], 'is_iceberg': true_test_predictions})

print(tlearn_submission.head(10))

tlearn_submission.to_csv('submissions/tlearn_submission.csv', index=False)

         id  is_iceberg
0  5941774d    0.007421
1  4023181e    0.352222
2  b20200e4    0.000630
3  e7f018bb    0.999915
4  4371c8c3    0.004748
5  a8d9b1fd    0.417789
6  29e7727e    0.036483
7  92a51ffb    0.999995
8  c769ac97    0.000078
9  aee0547d    0.000016
